In [34]:
# Imports:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [35]:
data = pd.read_csv('./train.csv')
data.head()

C:\Users\ayujv\AppData\Local\Temp\ipykernel_37396\356224728.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./train.csv')


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


## Data Preproccesing:
As you can see, we have a lot data to clean up and drop.

In [36]:
print(data.isnull().sum())

ID                              0
Customer_ID                     0
Month                           0
Name                         9985
Age                             0
SSN                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary       15002
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Changed_Credit_Limit            0
Num_Credit_Inquiries         1965
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1200
Credit_Score                    0
dtype: int64


In [37]:
# This is more specific than simply just using df.dtypes and helps us figure out why column 26 has mixed types.
for col in data.columns:
    print(f'{col} : {pd.api.types.infer_dtype(data[col])}')
  
# We need to one-hot-encode occupation, Type of Loans, Credit Mix, Payment of Minimum amount, Payment Behaviour
# We need to fix Age, Annual Income, Number of Loans, Number of Delayed Payments, 
#   Change in Credit Limit, Outstanding Debt, Credit History Age, Amount Invested Monthly, and Monthly Balance
# TODO: FOR EVERY NUMERICAL DATA TYPE, CHECK NEGATIVES AND HIGH OUTLIERS.

# Monthly_Inhand_Salary, Num_Bank_Accounts, Num_Credit_Card, Interest_Rate, Delay_from_due_date, Num_Credit_Inquiries, Credit_Utilization_Ratio, Total_EMI_per_month 
    

ID : string
Customer_ID : string
Month : string
Name : string
Age : string
SSN : string
Occupation : string
Annual_Income : string
Monthly_Inhand_Salary : floating
Num_Bank_Accounts : integer
Num_Credit_Card : integer
Interest_Rate : integer
Num_of_Loan : string
Type_of_Loan : string
Delay_from_due_date : integer
Num_of_Delayed_Payment : string
Changed_Credit_Limit : string
Num_Credit_Inquiries : floating
Credit_Mix : string
Outstanding_Debt : string
Credit_Utilization_Ratio : floating
Credit_History_Age : string
Payment_of_Min_Amount : string
Total_EMI_per_month : floating
Amount_invested_monthly : string
Payment_Behaviour : string
Monthly_Balance : mixed
Credit_Score : string


In [38]:
# Multiple types in column 26
column_data = data.iloc[ : , 26] 
print(column_data.apply(type).value_counts())

Monthly_Balance
<class 'str'>      97132
<class 'float'>     2868
Name: count, dtype: int64


In [39]:
# Drop useless features.
data.drop(['ID', 'Name', 'SSN', 'Month'], axis = 1, inplace = True)
data.head()

,Customer_ID,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,CUS_0xd40,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,CUS_0xd40,23,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,CUS_0xd40,-500,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,CUS_0xd40,23,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,CUS_0xd40,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [40]:
# Ensure that each customer has the same number of records before grouping them in the next steps.
(data.Customer_ID.value_counts() != 8).any()

False

In [41]:
# Convert 'Age' from string to float
data['Age'] = data['Age'].str.replace('_', '')
data['Age'] = data['Age'].replace("", np.nan)
data['Age'] = data['Age'].astype(float)

# Make all age values the same (the mode) for each customer.
grouped_data = data.groupby('Customer_ID')
mode_age = grouped_data['Age'].transform(lambda x: x.mode()[0]) # If there is a tie in mode (customer had a birthday), take the first mode.
data['Age'] = mode_age

print(data['Age'])
print(data['Age'].describe())
# Age now looks good.

0        23.0
1        23.0
2        23.0
3        23.0
4        23.0
         ... 
99995    25.0
99996    25.0
99997    25.0
99998    25.0
99999    25.0
Name: Age, Length: 100000, dtype: float64
count    100000.000000
mean         33.274560
std          10.764438
min          14.000000
25%          24.000000
50%          33.000000
75%          42.000000
max          56.000000
Name: Age, dtype: float64


In [42]:
# Annual income should change from a string to a float.
data['Annual_Income'] = data['Annual_Income'].str.replace('_', '')
data['Annual_Income'] = data['Annual_Income'].replace("", np.nan)
data['Annual_Income'] = data['Annual_Income'].astype(float)

# Make all Annual Income values the same (the mode) for each customer.
grouped_data = data.groupby('Customer_ID')
mode_ann_income = grouped_data['Annual_Income'].transform(lambda x: x.mode()[0]) # If there is a tie in Annual Income (customer had a promotion), take the first mode.
data['Annual_Income'] = mode_ann_income

print(data['Annual_Income'])
print(data['Annual_Income'].describe())
# Annual Income now looks good.

0        19114.12
1        19114.12
2        19114.12
3        19114.12
4        19114.12
           ...   
99995    39628.99
99996    39628.99
99997    39628.99
99998    39628.99
99999    39628.99
Name: Annual_Income, Length: 100000, dtype: float64
count    100000.000000
mean      50505.123449
std       38299.422093
min        7005.930000
25%       19342.972500
50%       36999.705000
75%       71683.470000
max      179987.280000
Name: Annual_Income, dtype: float64


In [43]:
# Number of loans should change from a string to a float.
data['Num_of_Loan'] = data['Num_of_Loan'].str.replace('_', '')
data['Num_of_Loan'] = data['Num_of_Loan'].replace("", np.nan)
data['Num_of_Loan'] = data['Num_of_Loan'].astype(float)

# Make all Annual Income values the same (the mode) for each customer.
grouped_data = data.groupby('Customer_ID')
mode_num_loan = grouped_data['Num_of_Loan'].transform(lambda x: x.mode()[0]) # If there is a tie in Annual Income (customer had a promotion), take the first mode.
data['Num_of_Loan'] = mode_num_loan
print(data['Num_of_Loan'].describe())


# negative_loans = data[data['Num_of_Loan'] < 0]['Num_of_Loan'].value_counts()
# print(negative_loans)
# # People can't have negative loans, so we drop these records.
# negative_loan_indices = data[data['Num_of_Loan'] < 0].index
# data = data.drop(negative_loan_indices)
# data.reset_index(drop = True, inplace = True)
# negative_loans = data[data['Num_of_Loan'] < 0]['Num_of_Loan'].any()
# print(negative_loans)
# # No more negative loans.

# num_loans = data["Num_of_Loan"].value_counts()
# print(num_loans)
# # Some people have 1000+ loans. We will determine outliers to be more than 10 loans.
# large_loan_indices = data[data['Num_of_Loan'] > 10].index
# data = data.drop(large_loan_indices)
# data.reset_index(drop = True, inplace = True)
# large_loans = data[data['Num_of_Loan'] > 10]['Num_of_Loan'].any()
# print(large_loans)
# Number of Loans now looks good.

count    100000.000000
mean          3.532880
std           2.446356
min           0.000000
25%           2.000000
50%           3.000000
75%           5.000000
max           9.000000
Name: Num_of_Loan, dtype: float64


In [44]:
# Number of delayed payments should change from a string to a float.
data['Num_of_Delayed_Payment'] = data['Num_of_Delayed_Payment'].str.replace('_', '')
data['Num_of_Delayed_Payment'] = data['Num_of_Delayed_Payment'].replace("", np.nan)
data['Num_of_Delayed_Payment'] = data['Num_of_Delayed_Payment'].astype(float)

# Make all Annual Income values the same (the mode) for each customer.
grouped_data = data.groupby('Customer_ID')
mode_num_loan = grouped_data['Num_of_Delayed_Payment'].transform(lambda x: x.mode()[0]) # If there is a tie in Annual Income (customer had a promotion), take the first mode.
data['Num_of_Delayed_Payment'] = mode_num_loan

negative_delayed_payments = data[data['Num_of_Delayed_Payment'] < 0]['Num_of_Delayed_Payment'].value_counts()
print(negative_delayed_payments)
# People can't have negative delayed payments, so we drop these records.
negative_delayed_payments = data[data['Num_of_Delayed_Payment'] < 0].index
data = data.drop(negative_delayed_payments)
data.reset_index(drop = True, inplace = True)
negative_delayed_payments = data[data['Num_of_Delayed_Payment'] < 0]['Num_of_Delayed_Payment'].any()
print(negative_delayed_payments)
# # No more negative delayed payments.

# num_delayed_payments = data["Num_of_Delayed_Payment"].value_counts()
# print(num_delayed_payments)
# # Some of these are in the thousands.
# print(data["Num_of_Delayed_Payment"].describe())
# # The 25th percentile is 9 and the 75th percentile is 18, so the IQR is 9. Outliers are anything above 18 + 1.5 * 9 = 31.5.
# large_delayed_payments = data[data['Num_of_Delayed_Payment'] > 31.5].index
# data = data.drop(large_delayed_payments)
# data.reset_index(drop = True, inplace = True)
# large_delayed_payments = data[data['Num_of_Delayed_Payment'] > 10]['Num_of_Delayed_Payment'].any()
# print(large_delayed_payments)
# print(data["Num_of_Delayed_Payment"].describe())

# # Handle NaNs
# print(data['Num_of_Delayed_Payment'].isnull().sum())
# data['Num_of_Delayed_Payment'].fillna(14, inplace = True)
print(data['Num_of_Delayed_Payment'].describe())
print(data['Num_of_Delayed_Payment'].isnull().sum())
# Number of Delayed Payments now looks good.

Num_of_Delayed_Payment
-2.0    64
-1.0    48
Name: count, dtype: int64
False
count    99888.000000
mean        13.280234
std          6.185304
min          0.000000
25%          9.000000
50%         14.000000
75%         18.000000
max         28.000000
Name: Num_of_Delayed_Payment, dtype: float64
0


In [45]:
# Change in credit limit should change from a string to a float.
data['Changed_Credit_Limit'] = data['Changed_Credit_Limit'].str.replace('_', '')
data['Changed_Credit_Limit'] = data['Changed_Credit_Limit'].replace("", np.nan)
data['Changed_Credit_Limit'] = data['Changed_Credit_Limit'].astype(float)
print(data['Changed_Credit_Limit'].isnull().sum())
# We have null values. Let's impute the mode for these for these values by grouping the records by their customer ID.
grouped_data = data.groupby('Customer_ID')
mode_changed_credit_limit = grouped_data['Changed_Credit_Limit'].transform(lambda x: x.mode()[0]) # Take the first mode.
data['Changed_Credit_Limit'] = mode_changed_credit_limit
print(data['Changed_Credit_Limit'].describe())
print(data['Changed_Credit_Limit'].isnull().sum())
# Change in credit limit now looks good. The mean is close to the median and negative values are okay and we have no null values.

2086
count    99888.000000
mean        10.398663
std          6.513531
min         -5.010000
25%          5.500000
50%          9.350000
75%         14.680000
max         29.980000
Name: Changed_Credit_Limit, dtype: float64
0


In [46]:
data['Outstanding_Debt'] = data['Outstanding_Debt'].str.replace('_', '')
data['Outstanding_Debt'] = data['Outstanding_Debt'].replace("", np.nan)
data['Outstanding_Debt'] = data['Outstanding_Debt'].astype(float)
grouped_data = data.groupby('Outstanding_Debt')
mode_outstanding_debt = grouped_data['Outstanding_Debt'].transform(lambda x: x.mode()[0]) # Take the first mode.
data['Outstanding_Debt'] = mode_outstanding_debt
print(data["Outstanding_Debt"].describe())
# Outstanding debt looks good. Although we have values near 5000 when the median is near 1170, it is reasonable that people have this much debt.

count    99888.000000
mean      1426.730355
std       1155.593285
min          0.230000
25%        566.080000
50%       1166.155000
75%       1948.980000
max       4998.070000
Name: Outstanding_Debt, dtype: float64


In [47]:
data['Amount_invested_monthly'] = data['Amount_invested_monthly'].str.replace('_', '')
data['Amount_invested_monthly'] = data['Amount_invested_monthly'].replace("", np.nan)
data['Amount_invested_monthly'] = data['Amount_invested_monthly'].astype(float)
grouped_data = data.groupby('Customer_ID')
mode_month_invested = grouped_data['Amount_invested_monthly'].transform(lambda x: x.mode()[0]) # If there is a tie in Annual Income (customer had a promotion), take the first mode.
data['Amount_invested_monthly'] = mode_month_invested
print(data["Amount_invested_monthly"].describe())
large_delayed_payments = data[data['Amount_invested_monthly'] >= 0]['Amount_invested_monthly'].value_counts()
print(large_delayed_payments)
# 10,000 seems to be a hard cap (i.e., if a record has more than $10,000 per month, set it to $10,000)

count    99888.000000
mean       505.787235
std       2048.242538
min          0.000000
25%         33.160175
50%         53.271704
75%         88.673028
max      10000.000000
Name: Amount_invested_monthly, dtype: float64
Amount_invested_monthly
10000.000000    4440
0.000000        1176
24.785217          8
13.965640          8
29.851579          8
                ... 
84.369190          8
71.786381          8
59.908904          8
51.910308          8
24.028477          8
Name: count, Length: 11786, dtype: int64


In [48]:
data['Monthly_Balance'] = data['Monthly_Balance'].str.replace('_', '')
data['Monthly_Balance'] = data['Monthly_Balance'].replace("", np.nan)
data['Monthly_Balance'] = data['Monthly_Balance'].astype(float)

negative_monthly_balance = data[data['Monthly_Balance'] < 0]['Monthly_Balance'].value_counts()
print(negative_monthly_balance)
# People can't have negative delayed payments, so we drop these records.
negative_monthly_balance = data[data['Monthly_Balance'] < 0].index
data = data.drop(negative_monthly_balance)
data.reset_index(drop = True, inplace = True)
negative_monthly_balance = data[data['Monthly_Balance'] < 0]['Monthly_Balance'].any()
print(negative_monthly_balance)

grouped_data = data.groupby('Customer_ID')
mode_month_balance = grouped_data['Monthly_Balance'].transform(lambda x: x.mean()) # USE MEAN B/C THIS CHANGES
data['Monthly_Balance'] = mode_month_balance
print(data["Monthly_Balance"].describe())

Monthly_Balance
-3.333333e+26    8
Name: count, dtype: int64
False
count    98184.000000
mean       402.562823
std        175.024662
min         92.841401
25%        284.373032
50%        338.297915
75%        464.875624
max       1349.264887
Name: Monthly_Balance, dtype: float64


In [49]:
# Monthly Inhand Salary is already a float.

negative_monthly_balance = data[data['Monthly_Inhand_Salary'] < 0]['Monthly_Inhand_Salary'].value_counts()
print(negative_monthly_balance)
# People can't have negative delayed payments, so we drop these records.
negative_monthly_balance = data[data['Monthly_Inhand_Salary'] < 0].index
data = data.drop(negative_monthly_balance)
data.reset_index(drop = True, inplace = True)
negative_monthly_balance = data[data['Monthly_Inhand_Salary'] < 0]['Monthly_Inhand_Salary'].any()
print(negative_monthly_balance)

grouped_data = data.groupby('Customer_ID')
mode_month_balance = grouped_data['Monthly_Inhand_Salary'].transform(lambda x: x.mode()[0]) # If there is a tie in Annual Income (customer had a promotion), take the first mode.
data['Monthly_Inhand_Salary'] = mode_month_balance
print(data["Monthly_Inhand_Salary"].describe())

Series([], Name: count, dtype: int64)
False
count    99880.000000
mean      4194.226734
std       3183.584409
min        303.645417
25%       1626.525000
50%       3091.000000
75%       5955.740000
max      15204.633333
Name: Monthly_Inhand_Salary, dtype: float64


In [50]:
data['Num_Bank_Accounts'] = data['Num_Bank_Accounts'].astype(float)

negative_monthly_balance = data[data['Num_Bank_Accounts'] < 0]['Num_Bank_Accounts'].value_counts()
print(negative_monthly_balance)
# People can't have negative delayed payments, so we drop these records.
negative_monthly_balance = data[data['Num_Bank_Accounts'] < 0].index
data = data.drop(negative_monthly_balance)
data.reset_index(drop = True, inplace = True)
negative_monthly_balance = data[data['Num_Bank_Accounts'] < 0]['Num_Bank_Accounts'].any()
print(negative_monthly_balance)

grouped_data = data.groupby('Customer_ID')
mode_month_balance = grouped_data['Num_Bank_Accounts'].transform(lambda x: x.mode()[0]) # If there is a tie in Annual Income (customer had a promotion), take the first mode.
data['Num_Bank_Accounts'] = mode_month_balance
print(data["Num_Bank_Accounts"].describe()) 

Num_Bank_Accounts
-1.0    21
Name: count, dtype: int64
False
count    99859.000000
mean         5.372125
std          2.590987
min          0.000000
25%          3.000000
50%          6.000000
75%          7.000000
max         10.000000
Name: Num_Bank_Accounts, dtype: float64


In [51]:
data['Num_Credit_Card'] = data['Num_Credit_Card'].astype(float)

negative_credit_card = data[data['Num_Credit_Card'] < 0]['Num_Credit_Card'].value_counts()
print(negative_credit_card)
# People can't have negative delayed payments, so we drop these records.
negative_credit_card = data[data['Num_Credit_Card'] < 0].index
data = data.drop(negative_credit_card)
data.reset_index(drop = True, inplace = True)
negative_credit_card = data[data['Num_Credit_Card'] < 0]['Num_Credit_Card'].any()
print(negative_credit_card)

grouped_data = data.groupby('Customer_ID')
mode_month_balance = grouped_data['Num_Credit_Card'].transform(lambda x: x.mode()[0]) # If there is a tie in Annual Income (customer had a promotion), take the first mode.
data['Num_Credit_Card'] = mode_month_balance
print(data["Num_Credit_Card"].describe())
# Interest_Rate, Delay_from_due_date, Num_Credit_Inquiries, Credit_Utilization_Ratio, Total_EMI_per_month 

Series([], Name: count, dtype: int64)
False
count    99859.000000
mean         5.534664
std          2.067365
min          0.000000
25%          4.000000
50%          5.000000
75%          7.000000
max         11.000000
Name: Num_Credit_Card, dtype: float64


In [52]:
data['Interest_Rate'] = data['Interest_Rate'].astype(float)

negative_credit_card = data[data['Interest_Rate'] < 0]['Interest_Rate'].value_counts()
print(negative_credit_card)
# People can't have negative delayed payments, so we drop these records.
negative_credit_card = data[data['Interest_Rate'] < 0].index
data = data.drop(negative_credit_card)
data.reset_index(drop = True, inplace = True)
negative_credit_card = data[data['Interest_Rate'] < 0]['Interest_Rate'].any()
print(negative_credit_card)

grouped_data = data.groupby('Customer_ID')
mode_month_balance = grouped_data['Interest_Rate'].transform(lambda x: x.mode()[0]) # If there is a tie in Annual Income (customer had a promotion), take the first mode.
data['Interest_Rate'] = mode_month_balance
print(data["Interest_Rate"].describe())
# Delay_from_due_date, Num_Credit_Inquiries, Credit_Utilization_Ratio, Total_EMI_per_month 

Series([], Name: count, dtype: int64)
False
count    99859.000000
mean        14.544257
std          8.740460
min          1.000000
25%          7.000000
50%         13.000000
75%         20.000000
max         34.000000
Name: Interest_Rate, dtype: float64


In [ ]:
data['Delay_from_due_date'] = data['Delay_from_due_date'].astype(float)

negative_credit_card = data[data['Delay_from_due_date'] < 0]['Delay_from_due_date'].value_counts()
print(negative_credit_card)
# People can't have negative delayed payments, so we drop these records.
negative_credit_card = data[data['Delay_from_due_date'] < 0].index
data = data.drop(negative_credit_card)
data.reset_index(drop = True, inplace = True)
negative_credit_card = data[data['Delay_from_due_date'] < 0]['Delay_from_due_date'].any()
print(negative_credit_card)

grouped_data = data.groupby('Customer_ID')
mode_month_balance = grouped_data['Interest_Rate'].transform(lambda x: x.mode()[0]) # If there is a tie in Annual Income (customer had a promotion), take the first mode.
data['Interest_Rate'] = mode_month_balance
print(data["Interest_Rate"].describe())
# Delay_from_due_date, Num_Credit_Inquiries, Credit_Utilization_Ratio, Total_EMI_per_month 